In [1]:
import re, textract, pickle, os

In [2]:
# STEP 1: Get file names

# in folder, click on tiny gear icon on top of the window, then select Copy "" pathname
path = '/Volumes/BackupWD2/CbCM data/Luxembourg Archive/2016_unlocked/2016_3/'
#path = '/home/op/kernel/thesis/data/cbm/pdfs/' # Oskar's path
# a package for navigating the operating system (just as from terminal)
# start with empty list
file_names = []
# iterate through all file names found in that path
for file in os.listdir(path):
    # append all files names to a list
    if file[0]!='.' and file[-3:]=='pdf': #to handle issue with hidden files and non-pdf files
        file_names.append(file)
file_names

['N°\xa00007-unlocked.pdf',
 'N°\xa00006-unlocked.pdf',
 'N°\xa00009-unlocked.pdf',
 'N°\xa00005-unlocked.pdf',
 'N°\xa00004-unlocked.pdf']

In [23]:
# Step 2: (can be skipped if pickle exists) Extract text from PDFs and save it to a pickle archive
extracted_pdfs = []
                  
for file_name in file_names: 
    text = str(textract.process(path+file_name))
    # each extracted text is saved in a dictionary under a key called file_name
    triple = [file_name, None, text] # just pack it all together
    extracted_pdfs.append(triple) # None is a placeholder for search results
    # after each extraction, overwrite a pickle archive
    print(file_name)
    pickle.dump(extracted_pdfs,open(path+'extracted_pdfs.pickle','wb'))

    

N° 0007-unlocked.pdf
N° 0006-unlocked.pdf
N° 0009-unlocked.pdf
N° 0005-unlocked.pdf
N° 0004-unlocked.pdf


In [24]:
# Step 3: Read from pickle that was just written
extracted_pdfs = pickle.load(open(path+'extracted_pdfs.pickle','rb'))
print('Number of loaded pdfs: ', len(extracted_pdfs))

Number of loaded pdfs:  5


In [25]:
# Step 4: Find the patterns
for i in range(0,len(extracted_pdfs)):#'projet de fusion'
    text = extracted_pdfs[i][2] # iterating through all triples i, fetch text
    
    ## change the search term to re-run the search on the pickle file.
    # List:
        # parts bénéficiaires,part bénéficiaires, parts beneficiaires, part beneficiaires, beneficial units, 
        # beneficial unit
        # projet de fusion
        #
    
    search = re.findall('projet de fusion', text) 
    # NOTE: this search result contains more interesting info https://developers.google.com/edu/python/regular-expressions
    extracted_pdfs[i][1]=len(search)
    print(extracted_pdfs[i][0], ' = ',len(search))
    pickle.dump(extracted_pdfs,open(path+'extracted_pdfs.pickle','wb'))
# sort the final list
from operator import itemgetter
extracted_pdfs = sorted(extracted_pdfs, key=itemgetter(1), reverse=True)
pickle.dump(extracted_pdfs,open(path+'extracted_pdfs.pickle','wb'))

N° 0007-unlocked.pdf  =  0
N° 0006-unlocked.pdf  =  0
N° 0009-unlocked.pdf  =  0
N° 0005-unlocked.pdf  =  0
N° 0004-unlocked.pdf  =  0


In [26]:
# Step 5: Copy files with more than 0 keywords to a new folder
new_folder_name = 'projet de fusion'
if os.path.exists(path+new_folder_name)==False:
    os.makedirs(path+new_folder_name)
    
from shutil import copyfile
for triple in extracted_pdfs:
    if triple[1]>0:
        copyfile(path+triple[0],path+new_folder_name+'/'+triple[0])